In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas
import numpy as np
import time

In [21]:
spark = SparkSession\
        .builder\
        .master("spark://192.168.2.148:7077") \
        .appName("Aram_StrongScaling_largerDATA")\
        .config("spark.dynamicAllocation.enabled", False)\
        .config("spark.driver.port", 9999)\
        .config("spark.blockManager.port", 10005)\
        .getOrCreate()

In [ ]:
schema = StructType([\
    StructField("TrackID", StringType(), True),\
    StructField("Genre", StringType(), True)])

million_song_df = spark.read.csv("hdfs://192.168.2.148:9000/user/ubuntu/SongCSVWithTrackID.csv", header=True, inferSchema=True)

genres_df = spark.read.option("delimiter", "\t").csv("hdfs://192.168.2.148:9000/user/ubuntu/genres.csv", schema=schema)

million_song_notbyte_df = million_song_df.withColumn("TrackID", regexp_replace(col("TrackID").cast(StringType()), "['b']", ""))


million_song_notbyte_df.take(1)
# Filter out songs with year=0
filtered_million_song_df = million_song_notbyte_df.filter(col("Year") != 0)

# Join the two DataFrames on the common column (e.g., track_id)
# joined_df = genres_df.join(filtered_million_song_df, genres_df["TrackID"] == million_song_df["TrackID"])
joined_df = genres_df.join(filtered_million_song_df, genres_df["TrackID"] == filtered_million_song_df["TrackID"])

joined_pandas = joined_df.toPandas()
joined_pandas.to_csv("Data_to_replicate.csv") #This is to be replciated to enlarge dataset. use "data_replicator.py"


In [ ]:
#Code for larger data set STRONG SCALING
million_song_df = spark.read.csv("hdfs://192.168.2.148:9000/user/ubuntu/replicated_Data_x2000.csv", header=True, inferSchema=True)
start_time = time.time()
joined_df = million_song_df.withColumn("TrackID", regexp_replace(col("TrackID").cast(StringType()), "['b']", ""))

# Group by genre and year, then calculate the percentage of songs in each genre by year
result_df = joined_df.groupBy("Genre", "Year").count()
result_df = result_df.orderBy("Year")
pivot_table = result_df.groupBy("Year").pivot("Genre").agg(count("Genre").alias("count")).fillna(0).orderBy("Year")

pivot_table = result_df


# Show the result or save it as needed
# filtered_million_song_df.take(1)
# joined_df.take(10)
# result_df.take(100)

# Group by Genre and Year and calculate the count
result_df = joined_df.groupBy("Genre", "Year").count()       

# Pivot the DataFrame to display counts for each genre in each year
pivot_result_df = result_df.groupBy("Year").pivot("Genre").agg(F.first("count"))

# Fill null values with 0
pivot_result_df    =    pivot_result_df.na.fill(0).orderBy("Year")

end_time = time.time()

print(f"Elapsed time: {end_time-start_time} seconds")

In [24]:
#WEAK SCALING
#change file to analyze manually ...x500... etc
million_song_df = spark.read.csv("hdfs://192.168.2.148:9000/user/ubuntu/replicated_Data_x2500.csv", header=True, inferSchema=True)
start_time = time.time()
joined_df = million_song_df.withColumn("TrackID", regexp_replace(col("TrackID").cast(StringType()), "['b']", ""))

# Group by genre and year, then calculate the percentage of songs in each genre by year
result_df = joined_df.groupBy("Genre", "Year").count()
result_df = result_df.orderBy("Year")
pivot_table = result_df.groupBy("Year").pivot("Genre").agg(count("Genre").alias("count")).fillna(0).orderBy("Year")

pivot_table = result_df

# Group by Genre and Year and calculate the count
result_df = joined_df.groupBy("Genre", "Year").count()       

# Pivot the DataFrame to display counts for each genre in each year
pivot_result_df = result_df.groupBy("Year").pivot("Genre").agg(F.first("count"))

# Fill null values with 0
pivot_result_df = pivot_result_df.na.fill(0).orderBy("Year")

end_time = time.time()

print(f"Elapsed time: {end_time-start_time} seconds")

Elapsed time: 20.502609491348267 seconds


In [ ]:
pivot_reslts_Pandas = pivot_result_df.toPandas()
pivot_reslts_Pandas = pivot_reslts_Pandas.set_index("Year")
# pivot_reslts_Pandas = pivot_reslts_Pandas.apply(lambda x: x*100/sum(x), axis=1)
pivot_reslts_Pandas_test = pivot_reslts_Pandas.div(pivot_reslts_Pandas.sum(axis=1), axis=0) * 100
interestingDF=pivot_reslts_Pandas_test[pivot_reslts_Pandas_test.index > 1970]
# interestingDF.plot(kind="bar", stacked=True, figsize=(20,20))

#save figure (will be on driver VM)
interestingDF.plot(kind="bar", stacked=True, figsize=(20,20)).get_figure().savefig("prelresult.png")

In [20]:
#release cores/stop app
spark.stop()